# Imports and Hyperparams


In [ ]:
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow as tf
import tensorflow_docs.vis.embed as embed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import io

tf.__version__

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-zgc8widi
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-zgc8widi
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0.dev0-py3-none-any.whl size=179891 sha256=78d0b8f6e8b5c5dcfec286d171872d84271bd39b970da100f2fc509896b47fc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-gun2rdtv/wheels/cc/c4/d8/5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


'2.8.0'

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 1
EPOCHS = 200
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([1, noise_dim])

# Connect to Drive

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


# Setup Project

In [ ]:
root = drive_path / 'MyDrive' / 'cs1430-final' # root folder of our project
reset = False # change this if you imported a new images.zip file

if not root.is_dir(): # create root folder
  %mkdir $root

%cd $root

data_dir = root / 'data'
res_dir = root / 'results'

if reset:
  %rm -rf $data_dir

# create the dirs in project setup

if not data_dir.is_dir(): 
  %mkdir $data_dir

if not res_dir.is_dir(): 
  %mkdir $res_dir

%ls

/content/drive/MyDrive/cs1430-final
102images.zip  data/  images.zip  results/  stylegan2-ada/


In [ ]:
import zipfile

if not (data_dir / 'images').is_dir(): # if we have not unzipped the images yet
  print("Extracting images...")

  with zipfile.ZipFile(str(root / 'images.zip'), 'r') as zip_ref: # unzip images into ./data/images
    zip_ref.extractall(str(data_dir))
    
  print('Extraction completed')
else:
  print("Images already exist!")

Extracting images...
Extraction completed


# Load Data

In [ ]:
import os
from skimage import io, img_as_float32

def load_image(path):
    return img_as_float32(io.imread(path))

In [ ]:


files = os.listdir(data_dir / 'images')
train_images = []
n = len(files)

for f in files:

  try: 
    img = io.imread('data/images/' + f)
    img = resize(img, (504, 504))
    train_images.append(img)
  except (IOError, SyntaxError) as e:
            print('Bad file:', f)

train_images = np.array(train_images)
train_images = train_images.reshape(train_images.shape[0], 504, 504, 3).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Streaming output truncated to the last 5000 lines.
Bad file: 019348-O Father O Satan O Sun!
Bad file: 019599-bury a friend
Bad file: 000294-Black WaveBad Vibrations
Bad file: 006726-This Year's Love
Bad file: 012967-Downtown's Dead
Bad file: 016591-Bom Bom - Radio Edit
Bad file: 015881-Heavydirtysoul
Bad file: 017208-I Go On
Bad file: 008030-End of the World
Bad file: 005717-One Way Glass
Bad file: 016061-Burning
Bad file: 018358-Where Is My Mind
Bad file: 001878-Lonely Loves
Bad file: 006975-Love Me Back
Bad file: 005163-Seventy Times 7
Bad file: 014106-You're a Wolf
Bad file: 009010-You Keep Hanging On
Bad file: 004540-The Kind
Bad file: 001005-Get Back - Anthology 3 Version
Bad file: 004507-Closer
Bad file: 010984-Bank Account
Bad file: 018462-Suppressant (Ft. Laura Brehm)
Bad file: 014011-Killer Queen - Remastered 2011
Bad file: 009527-Wow
Bad file: 006661-What the World Needs Now Is Love
Bad file: 016869-Awakening
Bad file: 010271-TiK ToK
Bad file: 016955-We Are the Kids
Bad file:

# Models

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 28, 28, 1)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 56, 56, 3)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (6, 6), strides=(9, 9), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 504, 504, 3)

    return model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[504, 504, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Loss and Optimizer

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Training

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal((BATCH_SIZE, noise_dim))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        print("Gen Loss: {} | Disc Loss: {}".format(gen_loss, disc_loss))

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
        train_step(image_batch)

    # Produce images for the GIF as you go
    # display.clear_output(wait=True)
    generate_and_save_images(generator, epoch + 1, seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0: checkpoint.save(file_prefix = checkpoint_prefix)
    print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    # display.clear_output(wait=True)
  generate_and_save_images(generator, epochs, seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure()
  img = predictions[0, :, :, :] * 127.5 + 127.5
  img = img / np.amax(img) # if float

  plt.imshow(img)
  plt.savefig(res_dir / 'image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
train(train_dataset, EPOCHS)

# Generate

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

display_image(EPOCHS)

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
embed.embed_file(anim_file)